In [11]:
import pandas as pd
import numpy as np
import glob
from sklearn.metrics.pairwise import cosine_similarity
import ast
import re

In [12]:
def extract_2d_array(tensor_str):
    
    tensor_str = tensor_str.replace('tensor(','').replace('grad_fn=<TanhBackward0>)', '').replace(' ', '').replace('\n', '')
    tensor_str = tensor_str[:len(tensor_str)-1]
    numbers = eval(tensor_str)
    
    array = np.array(numbers).reshape((-1, 768))
    
    return array

def extract_2d_array_list(data_str):

    data_str = data_str.replace('\n', ' ')

    # Replace one or more spaces with a single comma
    data_str = re.sub(r'\s+', ',', data_str)

    # Ensure there are no extra commas at the end
    data_str = '[' + data_str.strip(',') + ']'

    # Evaluate the string safely
    numbers = ast.literal_eval(data_str)

    array = np.array(numbers).reshape((-1, 768))  
    return array


In [13]:
def mean_of_2d_arrays(arrays):
    stacked_arrays = np.vstack(arrays)
    return np.mean(stacked_arrays, axis=0).reshape(-1, 768)

def findMeanEmbeddings(df):

    # Group by the specified column and calculate the mean of 2D arrays
    result = df.groupby('sub_category')['embeddings'].apply(lambda x: mean_of_2d_arrays(x))

    # Convert the result to a DataFrame
    result_df = pd.DataFrame(result)

    # Reset the index if necessary
    result_df.reset_index(inplace=True)
    return result_df

In [14]:
def constructAverageDataframe(path):
    files = glob.glob(path)
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        df = df.drop(columns=["Unnamed: 0", "Unnamed: 0.1", 'Unnamed: 0.2'], errors="ignore")
        dfs.append(df)
    
    df = pd.concat(dfs, ignore_index=True)
    df_unique = df.drop_duplicates(subset='name', keep='first')

    df_unique['embeddings'] = df_unique['embeddings'].apply(extract_2d_array)
    print(df_unique.shape)

    result_df = findMeanEmbeddings(df_unique)
    result_df.to_csv("average_embeddings.csv", index=False)

In [15]:
path = "./final_csvs/*/*.csv"
constructAverageDataframe(path)

/tmp/ipykernel_63207/2205712202.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['embeddings'] = df_unique['embeddings'].apply(extract_2d_array)


(59017, 3)


In [16]:
# def rankAvgDfBySimilarity(df, user_embedding):

#     df['embeddings'] = df['embeddings'].apply(lambda x: x.flatten())
#     embeddings_matrix = np.vstack(df['embeddings'].values)
#     print(embeddings_matrix.shape)

#     df['similarities'] = cosine_similarity(embeddings_matrix, user_embedding).flatten()
#     return df.sort_values(by='similarities', ascending=False).head(5)

# def getRandomEmbeddingVector(length):
#     vector = np.random.rand(length)
#     vector = 2 * vector - 1
#     return vector.reshape(1, -1)

# df = pd.read_csv("average_embeddings.csv")
# df['embeddings'] = df['embeddings'].apply(extract_2d_array_list)

# df = rankAvgDfBySimilarity(df,getRandomEmbeddingVector(768))
# df

In [17]:
# categories_list = df['sub_category']
# print(categories_list)
# hidden_df = pd.read_csv("dataset_80000.csv")

# final_df = []

# for category in categories_list:
#     print(category)
#     df_filtered = hidden_df[hidden_df['sub_category'] == category]
#     final_df.append(df_sampled)

# recommendations = pd.concat(final_df, ignore_index=True)

# print(recommendations)